# VISUALIZATION WITH TensorBoard

* View learning curves during training
* Compare learning curves between multiple runs
* Visualize the computation graph
* Visualize complex multidimensional data projected down to 3D and automatically clustered for you
* etc.

Installed automatically with TF

* Modify your program to output the data to visualize to special binary log files: **event files**
* Each binary data record called a **summary**

## Create a TensorBoard callback

In [1]:
# Define root log directory

import os
root_logdir = os.path.join(os.curdir, 'my_logs')

In [2]:
# Generate a subdirectory path based on current date and time: different name at each run
# can add extra info such as hyperparameter values...

def get_run_logdir():
    import time
    run_id = time.strftime('run-%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()  # ex: './my_logs/run-2020_10_30-15_21_34'

In [ ]:
# Create and `TensorBoard()` callback
# Will create one log directory per run (subdirs for training and validation logs)

# ... build and compile your model
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train,
                    epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

## Start the TensorBoard server

* Run following command in the terminal:
  ```
  $ tensorboard --logdir=./my_logs --port=6006
  TensorBoard 2.9.9 at http://mycomputer.local:6006/ (Press Ctrl+C to quit)
  ```

* Then:
  * **In your browser**, go to  
    ```
    http://localhost:6006
    ```
  * **Or within Jupyter**, run the following commands:
    ```
    %load_ext tensorboard
    %tensorboard --logdir=./my_logs port=6006
    ```

## More with the `tf.summary` package

Log scalars, histograms, images, audio and text to visualize them in TensorBoard

In [ ]:
test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)

with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar('my_scalar', np.sin(step / 10), step=step)
        data = (np.random.randn(100) + 2) * step / 100                 # some random data
        tf.summary.histogram('my_hist', data, buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3)                          # random 32 by 32 RGB images
        tf.summary.image('my_images', images * step / 100, step=step)
        texts = ['The step is ' + str(step), "Its square is " + str(step**2)]
        tf.summary.text('my_text', texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio('my_audio', audio, sample_rate=48000, step=step)